Thanks to Chris: https://www.kaggle.com/c/lish-moa/discussion/195195

In [ ]:
SEED = 42
FOLDS = 5

# LOAD LIBRARIES (from PIP or Kaggle Dataset)
! pip install iterative-stratification 
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import pandas as pd


In [ ]:

# LOAD FILES
scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
targets = scored.columns[1:]
scored = scored.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
scored['kfold'] = scored.drug_id.map(dct1)
scored.loc[scored.kfold.isna(),'kfold'] =\
    scored.loc[scored.kfold.isna(),'sig_id'].map(dct2)
scored.kfold = scored.kfold.astype('int8')

In [ ]:
scored[['sig_id', 'kfold']].to_csv('xfolds.csv', index = False)